In [32]:
import torch
import torch.optim as optim
from torch import nn
from torch.utils.data import DataLoader, random_split
from gloss_dataset import GlossDataset
from gloss_model import GlossModel
from torch.utils.tensorboard import SummaryWriter


In [33]:
# get data for training
gd = GlossDataset()
input_size = gd[0][0].shape[1]
class_no = len(gd.classes)
input_size, class_no

(1596, 4)

In [34]:
# provide input and class size
model = GlossModel(input_size, class_no)
optim = optim.Adam(model.parameters(), lr=1e-4)
loss_fn = nn.CrossEntropyLoss()
model


GlossModel(
  (softmax): Softmax(dim=1)
  (relu): ReLU()
  (lstm1): LSTM(1596, 128, batch_first=True)
  (dropout1): Dropout(p=0.15, inplace=False)
  (lstm2): LSTM(128, 64, batch_first=True)
  (dropout2): Dropout(p=0.15, inplace=False)
  (fc1): Linear(in_features=64, out_features=32, bias=True)
  (dropout3): Dropout(p=0.15, inplace=False)
  (fc2): Linear(in_features=32, out_features=4, bias=True)
)

In [35]:
# initialize sumamry writer
writer=SummaryWriter()

In [36]:
# Create testing and training dataLoader from single dataset using random_split
# and also set training epoch
split_ratio = 0.8
batch_size = 3
train_size = int(split_ratio*len(gd))
test_size = len(gd)-train_size
train_data, test_data = random_split(gd, [train_size, test_size])
train_dl = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_dl = DataLoader(test_data, batch_size=batch_size, shuffle=True)
epoch = 4000


In [37]:
# Start model training
model.train()
for i in range(epoch+1):
    for x_train, y_train in train_dl:
        optim.zero_grad()
        out = model(x_train)
        loss = loss_fn(out, y_train)
        loss.backward()
        optim.step()
    writer.add_scalar("Loss/epoch", loss.item(), i)
    if (i % 100 == 0):
        print(f"Loss/epoch : {loss.item(),i}")
    writer.close()


Loss/epoch : (3.87121844291687, 0)
Loss/epoch : (3.7425785064697266, 100)
Loss/epoch : (3.1884632110595703, 200)
Loss/epoch : (3.1667633056640625, 300)
Loss/epoch : (3.1616437435150146, 400)
Loss/epoch : (3.158705711364746, 500)
Loss/epoch : (3.1590282917022705, 600)
Loss/epoch : (3.1574461460113525, 700)
Loss/epoch : (3.15726637840271, 800)
Loss/epoch : (3.1575164794921875, 900)
Loss/epoch : (3.157027006149292, 1000)
Loss/epoch : (3.157036542892456, 1100)
Loss/epoch : (3.156921625137329, 1200)
Loss/epoch : (3.156505584716797, 1300)
Loss/epoch : (3.1566762924194336, 1400)
Loss/epoch : (3.156435012817383, 1500)
Loss/epoch : (3.15664005279541, 1600)
Loss/epoch : (3.156458616256714, 1700)
Loss/epoch : (3.1564083099365234, 1800)
Loss/epoch : (3.1564056873321533, 1900)
Loss/epoch : (3.156407117843628, 2000)
Loss/epoch : (3.1564228534698486, 2100)
Loss/epoch : (3.156440496444702, 2200)
Loss/epoch : (3.1563901901245117, 2300)
Loss/epoch : (3.1563761234283447, 2400)
Loss/epoch : (3.15638566017

In [38]:
# Evaluate Test set
model.eval()
tot_loss = 0
tp = 0
tot_test = 0
with torch.no_grad():
    for x_test, y_test in test_dl:
        out = model(x_test)
        loss = loss_fn(out, y_train)
        tot_loss += loss
        y_pred = torch.argmax(out[2])
        y_test = torch.argmax(y_test)
        if y_pred == y_test:
            tp += 1
        tot_test += 1

# Much more data required for training and test set
acc = tp/tot_test
loss = tot_loss/tot_test
print(f"Test Acc {acc}\nTest Loss : {loss}\n")


Test Acc 1.0
Test Loss : 3.1563732624053955



In [39]:
# save the model
torch.save(model, "swaram_lstm.pt")
